In [1]:
import pandas as pd
from omb.backend import dataset
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
def _cell_type_name_to_cell_ids(cell_type_name, sample=None):
    cell_clusters = dataset.get_variables(
        dataset.cluster_name_to_level[cell_type_name])
    cell_clusters = cell_clusters[cell_clusters == cell_type_name]
    if (sample is not None) and (cell_clusters.size > sample):
        cell_clusters = cell_clusters.sample(sample, random_state=1)

    cell_ids = cell_clusters[cell_clusters == cell_type_name].index
    return cell_ids


In [3]:
cell_type_name = 'PT-L5'

In [4]:
def _prepare_for_both_scatter(coord_and_cell_type_level, cell_type_name):
    try:
        coord_base, cell_type_level = coord_and_cell_type_level.split(' - ')
    except ValueError:
        raise PreventUpdate

    selected_cells = _cell_type_name_to_cell_ids(cell_type_name)

    selected_plot_df, unselected_plot_df, hover_cols, palette = get_split_plot_df(
        coord_base=coord_base,
        variable_name=cell_type_level,
        selected_cells=selected_cells,
        hover_cols=('RegionName', 'SubType'))

    if selected_plot_df.shape[0] > DOWN_SAMPLE:
        selected_plot_df = selected_plot_df.sample(DOWN_SAMPLE, random_state=0)
    if unselected_plot_df.shape[0] > DOWN_SAMPLE:
        unselected_plot_df = unselected_plot_df.sample(DOWN_SAMPLE, random_state=0)
    return selected_plot_df, unselected_plot_df, cell_type_level, palette, hover_cols


In [5]:
annoj_meta = pd.read_csv('AnnoJMeta.csv', index_col=0)
gene_track_id = annoj_meta.loc['Gene', 'id']
subtype_to_annoj_track_id = annoj_meta[annoj_meta['type'] == 'MethTrack']['id'].to_dict()
annoj_meta.head()

,id,type,path,data,iconCls,height,scale,showControls,boxWidthMin,single,showWalks,class,policy
name,,,,,,,,,,,,,
Gene,1_1,ModelsTrack,Annotation Models,/aj2/models/mm15-simple.php,silk_bricks,100,1,1.0,NaN,NaN,NaN,NaN,NaN
Vista Enhancers,1_2,ModelsTrack,Annotation Models,/aj2/tracks/hchen/mm_chol/Annotation_Models/Vi...,silk_bricks,28,1,NaN,1.0,1.0,1.0,NaN,NaN
CA1 Ptprg,2_1,MethTrack,Excitatory Neuron/CA1,/aj2/tracks/hchen/mouse_brain_161/mC/CA1_Ptprg...,salk_meth,40,1,NaN,NaN,NaN,NaN,CG CH -coverage,1:1:10
CA1 Lingo2,2_2,MethTrack,Excitatory Neuron/CA1,/aj2/tracks/hchen/mouse_brain_161/mC/CA1_Lingo...,salk_meth,40,1,NaN,NaN,NaN,NaN,CG CH -coverage,1:1:10
CA1 Chrm3,2_3,MethTrack,Excitatory Neuron/CA1,/aj2/tracks/hchen/mouse_brain_161/mC/CA1_Chrm3...,salk_meth,40,1,NaN,NaN,NaN,NaN,CG CH -coverage,1:1:10


In [6]:
url = dataset.annoj_url(active_clusters=['IT-L23', 'IT-L4', 'IT-L5', 'IT-L6'],
                        chrom='2',
                        start=10000000,
                        end=10100000,
                        track_type='CH',
                        mc_track_height=50,
                        hide_sidebar=True,
                        hide_toolbar=False,
                        cell_type_color=True)
print(url)

http://neomorph.salk.edu/mouse_brain.php?location=2:10000000-10100000&active=["12_2","12_4","12_3","12_1","13_2","13_1","14_2","14_1","14_3","15_2","15_3","15_1","15_4"]&config=[{type:"MethTrack",height:50,class:"-CG CH -coverage"},{id:"12_2",color:{CH:"C8CA86",rColor:1}},{id:"12_3",color:{CH:"475400",rColor:1}},{id:"12_4",color:{CH:"94B400",rColor:1}},{id:"12_1",color:{CH:"212900",rColor:1}},{id:"13_2",color:{CH:"A8D617",rColor:1}},{id:"13_1",color:{CH:"B3D164",rColor:1}},{id:"14_3",color:{CH:"435F00",rColor:1}},{id:"14_1",color:{CH:"A6D648",rColor:1}},{id:"14_2",color:{CH:"203900",rColor:1}},{id:"15_4",color:{CH:"6DC600",rColor:1}},{id:"15_3",color:{CH:"42A100",rColor:1}},{id:"15_1",color:{CH:"3BB800",rColor:1}},{id:"15_2",color:{CH:"167600",rColor:1}}]&settings={accordion:"hide",}
